In [1]:
LSP_ANNOTATION_FILE = '/home/ubuntu/datasets/lsp/CO_LSP_train2016.json'
LSP_IMAGES_FOLDER = 'http://vision.caltech.edu/~mronchi/data/LSP/images/'
LSP_IMAGES_SERVER_FOLDER = '/static/images/lsp/'

with open( LSP_ANNOTATION_FILE ) as f:
    _lsp_dataset = json.load(f)

IOError: [Errno 2] No such file or directory: '/home/ubuntu/datasets/lsp/CO_LSP_train2016.json'